In [18]:
import pyfaidx
import os 
import pandas as pd 
from tqdm import tqdm

path_to_all_fa = "/Volumes/HNSD01/storage/ref/hg19" 
    
def get_refseq(path_to_all_fa, chrom, start, end):
    
    refseq = pyfaidx.Fasta(os.path.join(path_to_all_fa, "{}.fa".format(chrom)))
    return(str.upper(refseq.get_seq(name = "{}".format(chrom), start = start, end = end).seq))

def find_CpG(ref_seq):
    import re
    all_C_positions = [m.start(0) for m in re.finditer("CG", ref_seq)]
    all_c_positions = [m.start(0) for m in re.finditer("cg", ref_seq)]
    all_cG_positions = [m.start(0) for m in re.finditer("cG", ref_seq)]
    all_Cg_positions = [m.start(0) for m in re.finditer("Cg", ref_seq)]
    return all_C_positions + all_c_positions + all_cG_positions + all_Cg_positions

tsma_regiondf = pd.read_excel("./assets/TSMA panel.xlsx", skiprows=2)
tsma_regiondf.columns = ["chrom", "start", "end", "annotation", "panel"]
tsma_regiondf["panel"] = tsma_regiondf["panel"].apply(lambda x: x.replace(" ", "_"))
tsma_regiondf["regionname"] = tsma_regiondf[["chrom", "start", "end"]].apply(lambda x: f"chr{x[0]}_{x[1]}_{x[2]}", axis = 1)

df = pd.DataFrame()
for panel in tsma_regiondf["panel"].unique():
    regions = tsma_regiondf[tsma_regiondf["panel"] == panel]["regionname"].unique()
    for region in tqdm(regions):
        region_chrom = region.split("_")[0]
        region_start = int(region.split("_")[1])
        region_end = int(region.split("_")[2])
        refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                            chrom = region_chrom, 
                                            start = region_start, 
                                            end = region_end + 1)
        all_cpg_in_cluster = sorted(find_CpG(refseq_at_cluster))
        cpg_coords = [item + region_start for item in all_cpg_in_cluster]
        tmpdf = pd.DataFrame(data = cpg_coords, columns = ["pos"])
        tmpdf["chrom"] = region_chrom
        tmpdf["region"] = region
        tmpdf["panel"] = panel
        df = pd.concat([df, tmpdf], axis = 0)
    
df["start"] = df["pos"].copy()
df["end"] = df["start"] + 1
df = df[["chrom", "start", "end", "region", "panel"]]

df.to_csv("all_cpgs_in_all_panels.csv", index = False)

100%|██████████| 451/451 [00:00<00:00, 1050.59it/s]
